In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os,sys
import scanpy as sc 
import pandas as pd
import numpy as np
# import milopy
import scipy
import anndata

In [4]:
# check X stores raw counts 
def _check_counts_in_X(adata):
    return(all(np.random.choice(adata.X.data, 100) % 1 == 0))

def _clean_adata(a):
    ## Make obs_names unique
    a.obs_names = a.obs['dataset_id'].astype('str') + '-' + a.obs_names.astype("str")
    assert _check_counts_in_X(a)

    sc.pp.calculate_qc_metrics(a, inplace=True)
    sc.pp.filter_cells(a, min_counts=1000)
    return(a)

### Prep COVID dataset

In [7]:
h5ad_files_covid = '/Users/lianama/CIR/contrastive_inverse_regression/examples/PMBCs_2/data/PBMC_CZI_integration_filtered/PBMC_COVID.subsample500cells.covid.h5ad'

adata_covid = sc.read_h5ad(h5ad_files_covid)
cleaned_covid = _clean_adata(adata_covid)


### Prep Control Dataset

In [6]:
h5ad_files_ctrl = '/Users/lianama/CIR/contrastive_inverse_regression/examples/PMBCs_2/data/PBMC_CZI_integration_filtered/PBMC_COVID.subsample500cells.ctrl.h5ad'

adata_ctrl = sc.read_h5ad(h5ad_files_ctrl)
cleaned_ctrl = _clean_adata(adata_ctrl)

rows represent individual cells
columns represent genes 

In [8]:
expression_matrix_covid = cleaned_covid.X
annotations_covid = cleaned_covid.obs   # for observations (e.g., cells)
var_annotations_covid = cleaned_covid.var   # for variables (e.g., genes)

In [9]:
# Access other metadata
metadata_covid = cleaned_covid.uns 

In [10]:
print(expression_matrix_covid)

  (0, 75)	1.0
  (0, 97)	1.0
  (0, 110)	1.0
  (0, 161)	1.0
  (0, 205)	1.0
  (0, 214)	1.0
  (0, 247)	1.0
  (0, 314)	1.0
  (0, 325)	1.0
  (0, 339)	2.0
  (0, 366)	1.0
  (0, 389)	1.0
  (0, 393)	2.0
  (0, 416)	1.0
  (0, 438)	1.0
  (0, 455)	1.0
  (0, 458)	1.0
  (0, 504)	1.0
  (0, 542)	1.0
  (0, 586)	1.0
  (0, 614)	1.0
  (0, 697)	1.0
  (0, 700)	2.0
  (0, 718)	1.0
  (0, 774)	1.0
  :	:
  (48082, 24522)	1.0
  (48082, 24524)	1.0
  (48082, 24545)	3.0
  (48082, 24558)	1.0
  (48082, 24580)	1.0
  (48082, 24587)	1.0
  (48082, 24588)	4.0
  (48082, 24614)	1.0
  (48082, 24629)	1.0
  (48082, 24662)	2.0
  (48082, 24663)	8.0
  (48082, 24691)	1.0
  (48082, 24699)	1.0
  (48082, 24700)	9.0
  (48082, 24701)	3.0
  (48082, 24702)	24.0
  (48082, 24703)	13.0
  (48082, 24704)	2.0
  (48082, 24705)	4.0
  (48082, 24706)	10.0
  (48082, 24707)	2.0
  (48082, 24708)	3.0
  (48082, 24709)	1.0
  (48082, 24710)	2.0
  (48082, 24712)	1.0


In [2]:
# expression_matrix_covid_dense = expression_matrix_covid.todense()
# print(expression_matrix_covid_dense)

In [11]:
print(annotations_covid)

                                                     sex tissue ethnicity  \
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
...                                                  ...    ...       ...   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   

                                                     disease  \
10_1038_s41

In [12]:
print(var_annotations_covid)

                feature_biotype  n_cells_by_counts  mean_counts  \
feature_id                                                        
ENSG00000243485            gene                  0     0.000000   
ENSG00000238009            gene                 39     0.000811   
ENSG00000239945            gene                  0     0.000000   
ENSG00000239906            gene                  0     0.000000   
ENSG00000229905            gene                  0     0.000000   
...                         ...                ...          ...   
ENSG00000277196            gene                  2     0.000042   
ENSG00000278384            gene                123     0.002641   
ENSG00000277856            gene                187     0.189672   
ENSG00000275063            gene                235     0.129859   
ENSG00000271254            gene                434     0.009234   

                 log1p_mean_counts  pct_dropout_by_counts  total_counts  \
feature_id                                           

In [15]:
print(metadata_covid)

OrderedDict()


In [17]:
outdir = '/Users/lianama/CIR/contrastive_inverse_regression/examples/PMBCs_2/PBMC_COVID/'
adata_covid.obs['donor_id'] = adata_covid.obs['donor_id'].astype("str").astype("category")
adata_ctrl.obs['donor_id'] = adata_ctrl.obs['donor_id'].astype("str").astype("category")

In [18]:
adata_covid.write_h5ad(outdir + 'PBMC_COVID.subsample500cells.covid.h5ad')
adata_ctrl.write_h5ad(outdir + 'PBMC_COVID.subsample500cells.ctrl.h5ad')